In [1]:
import sys
import os

project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))

if project_root not in sys.path:
    sys.path.insert(0, project_root)

from evaluation.eval_utils import *
from evaluation.eval_utils import *
import json
#from retrieval import *
import numpy as np
import faiss
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import word_tokenize
from rank_bm25 import BM25Okapi

In [2]:


#########################################
# Retrieval Method Functions
#########################################

def tfidf_retrieval(query, config):
    """
    TF-IDF retrieval.
    config must include:
      - "vectorizer": a fitted TfidfVectorizer,
      - "doc_matrix": document-term matrix,
      - "passages": list of passages,
      - "chunk_ids": list of passage identifiers,
      - "top_k": number of top results.
    """
    vectorizer = config["vectorizer"]
    doc_matrix = config["doc_matrix"]
    passages = config["passages"]
    chunk_ids = config["chunk_ids"]
    top_k = config.get("top_k", 5)
    
    query_vec = vectorizer.transform([query])
    cosine_similarities = (doc_matrix @ query_vec.T).toarray().flatten()
    sorted_indices = np.argsort(cosine_similarities)[::-1][:top_k]
    
    results = [{"chunk_id": chunk_ids[i], "score": float(cosine_similarities[i])} 
               for i in sorted_indices]
    return results

def bm25_retrieval(query, config):
    """
    BM25 retrieval.
    config must include:
      - "bm25": a BM25Okapi object,
      - "passages": list of passages,
      - "chunk_ids": list of passage identifiers,
      - "top_k": number of top results.
    """
    bm25 = config["bm25"]
    passages = config["passages"]
    chunk_ids = config["chunk_ids"]
    top_k = config.get("top_k", 5)
    
    tokenized_query = word_tokenize(query.lower())
    scores = bm25.get_scores(tokenized_query)
    sorted_indices = np.argsort(scores)[::-1][:top_k]
    
    results = [{"chunk_id": chunk_ids[i], "score": float(scores[i])} 
               for i in sorted_indices]
    return results



def dense_retrieval_subqueries(queries, config):

    if isinstance(queries, str):
        queries = [queries]

    results = []
    top_k = config.get("top_k", 5)
    for query in queries:
        query_emb = query_embed_search(query, config["all_subqueries"], config["subquery_index"])
        query_emb = query_emb.reshape(1, -1)  # Reshape to (1, d)
        distances, indices = config["faiss_index"].search(query_emb, top_k)
        results.extend([
            {
                "sub_query": query,
                "chunk_id": config["chunk_ids"][i],
                "passage": config["passages"][i],
                "score": float(distances[0][j])
            } for j, i in enumerate(indices[0])
        ])
    return results

def query_embed_search(query, all_queries_list, index):
    """
    Given a query, finds its embedding from the precomputed subqueries index.
    """
    try:
        position = all_queries_list.index(query)
        return index.reconstruct(position)
    except ValueError:
        raise ValueError(f"Query '{query}' not found in the list of all queries.")
    

def hybrid(query, config):
    """
    Hybrid retrieval: first retrieve candidates with one method, then re-rank with the other.
    
    The order is determined by config["order"]:
      - "sparse_dense" (default): Use query[0] (or query if not a list) for sparse retrieval,
                                   then query[1] (or query) for dense retrieval.
      - "dense_sparse": Use query[0] for dense retrieval,
                        then query[1] for sparse retrieval.
    
    The candidate sets are combined (using the "mode": "intersection" or "union")
    and then sorted by the score of the re-ranking model.
    
    Parameters:
      - query (str or list): If a list of at least two elements, the two parts will be used
                             in the order specified by config["order"]. Otherwise, the same query is used.
      - config (dict): Must contain:
          * "order": either "sparse_dense" (default) or "dense_sparse"
          * "sparse_func": sparse retrieval function (e.g., tfidf_retrieval or bm25_retrieval)
          * "sparse_config": configuration for the sparse method.
          * "dense_func": dense retrieval function (e.g., dense_retrieval_subqueries)
          * "dense_config": configuration for the dense method.
          * "intermediate_k": candidate set size (default: 30)
          * "final_k": final result count (default: 5)
          * "mode": "intersection" (default) or "union"
    
    Returns:
      A list of final results (dictionaries) from the re-ranking step.
    """
    order = config.get("order", "sparse_dense")
    intermediate_k = config.get("intermediate_k", 30)
    final_k = config.get("final_k", 5)
    mode = config.get("mode", "intersection")
    
    # Decide which query part to use for each retrieval step.
    if isinstance(query, list) and len(query) >= 2:
        query_sparse = query[0]
        query_dense = query[1]
    else:
        print("Query should be a list of two queries: [sparse_query, dense_query].")


    if order == "sparse_dense":
        # Sparse retrieval first.
        sparse_config = config["sparse_config"].copy()
        sparse_config["top_k"] = intermediate_k
        sparse_results = config["sparse_func"](query_sparse, sparse_config)
        
        # Dense retrieval.
        dense_results = config["dense_func"](query_dense, config["dense_config"])
        
        # Combine: here we choose to filter dense results by the candidate set from sparse.
        candidate_ids = set(r["chunk_id"] for r in sparse_results)
        filtered_dense = [r for r in dense_results if r["chunk_id"] in candidate_ids]
        if not filtered_dense:
            filtered_dense = dense_results
        # Sort by dense score.
        sorted_results = sorted(filtered_dense, key=lambda x: x["score"], reverse=True)
    
    elif order == "dense_sparse":
        # Dense retrieval first.
        dense_results = config["dense_func"](query_dense, config["dense_config"])
        # Sparse retrieval.
        sparse_config = config["sparse_config"].copy()
        sparse_config["top_k"] = intermediate_k
        sparse_results = config["sparse_func"](query_sparse, sparse_config)
        # Combine: filter sparse results by dense candidate set.
        candidate_ids = set(r["chunk_id"] for r in dense_results)
        filtered_sparse = [r for r in sparse_results if r["chunk_id"] in candidate_ids]
        if not filtered_sparse:
            filtered_sparse = sparse_results
        # Sort by sparse score.
        sorted_results = sorted(filtered_sparse, key=lambda x: x["score"], reverse=True)
    else:
        raise ValueError("Invalid order specified.")
    
    return sorted_results[:final_k]
def hybrid(query, config):
    """
    Hybrid retrieval: first uses a sparse retrieval method to obtain an intermediate candidate set,
    then re-ranks these candidates using a dense retrieval function, and optionally re-ranks again
    using a cross encoder.

    The function expects the input query to be either:
      - a single string (in which case the same query is used for both steps), or
      - a list with at least two elements, where:
           query[0] is used for the sparse retrieval step and
           query[1] is used for the dense retrieval step.
    If query[0] is a list, it will be joined into a single string.

    Additionally, if config["rerank"] is True and a "cross_encoder_model" is provided,
    the final candidates will be re-ranked by the cross encoder.

    Parameters:
      - query (str or list): The input query.
      - config (dict): A configuration dictionary containing:
            * "sparse_func": sparse retrieval function (e.g., tfidf_retrieval or bm25_retrieval)
            * "sparse_config": configuration for the sparse retrieval method.
            * "dense_func": dense retrieval function (e.g., dense_retrieval_subqueries)
            * "dense_config": configuration for the dense retrieval method.
            * "intermediate_k": number of candidates to retrieve using the sparse method (default: 30)
            * "final_k": number of final results to return after dense re-ranking (default: 5)
            * "mode": "intersection" (default) or "union" for candidate filtering.
            * Optional: "rerank": True/False to perform cross encoder re-ranking.
            * Optional: "cross_encoder_model": a CrossEncoder instance from SentenceTransformers.
    
    Returns:
      A list of dictionaries corresponding to the final top-k results.
    """
    if isinstance(query, list) and len(query) >= 2:
        # For sparse retrieval, join if needed.
        if isinstance(query[0], list):
            query_sparse = " ".join(query[0])
        else:
            query_sparse = query[0]
        # For dense retrieval, also join if it's a list.
        query_dense = query[1]
        if isinstance(query_dense, list):
            query_dense = " ".join(query_dense)
    else:
        query_sparse = query
        query_dense = query

    intermediate_k = config.get("intermediate_k", 30)
    final_k = config.get("final_k", 5)
    mode = config.get("mode", "intersection")
    
    # Step 1: Sparse retrieval using query_sparse.
    sparse_config = config["sparse_config"].copy()
    sparse_config["top_k"] = intermediate_k
    sparse_results = config["sparse_func"](query_sparse, sparse_config)
    
    # Step 2: Dense retrieval using query_dense.
    dense_results = config["dense_func"](query_dense, config["dense_config"])
    
    # Step 3: Combine candidate sets.
    if mode == "intersection":
        candidate_ids = set(r["chunk_id"] for r in sparse_results)
        filtered_dense = [r for r in dense_results if r["chunk_id"] in candidate_ids]
        if not filtered_dense:
            filtered_dense = dense_results  # fallback if intersection is empty
    elif mode == "union":
        candidate_ids = set(r["chunk_id"] for r in sparse_results).union(
                        set(r["chunk_id"] for r in dense_results))
        dense_dict = {r["chunk_id"]: r for r in dense_results}
        filtered_dense = []
        for cid in candidate_ids:
            if cid in dense_dict:
                filtered_dense.append(dense_dict[cid])
            else:
                filtered_dense.append({"chunk_id": cid, "score": 0.0})
    else:
        raise ValueError("Invalid mode. Choose 'intersection' or 'union'.")
    
    # Step 4: Sort by dense score (descending) and select top final_k.
    sorted_dense = sorted(filtered_dense, key=lambda x: x["score"], reverse=True)
    final_results = sorted_dense[:final_k]
    
    # Step 5 (Optional): Cross-Encoder re-ranking.
    if config.get("rerank", False) and "cross_encoder_model" in config:
        cross_encoder = config["cross_encoder_model"]
        # Before passing sentence pairs to cross_encoder.predict, ensure each passage is a string.
        sentence_pairs = []
        for candidate in final_results:
            passage = candidate.get("passage", "")
            if not isinstance(passage, str):
                passage = str(passage)
            sentence_pairs.append([query_dense, passage])
        try:
            cross_scores = cross_encoder.predict(sentence_pairs)
        except Exception as e:
            print("Error during cross encoder prediction:", e)
            cross_scores = [candidate["score"] for candidate in final_results]  # fallback
        for idx, candidate in enumerate(final_results):
            candidate["cross_encoder_score"] = float(cross_scores[idx])
        final_results = sorted(final_results, key=lambda x: x["cross_encoder_score"], reverse=True)
    
    return final_results

#########################################
# Dataset Configuration
#########################################

# Global corpus (for all retrieval methods that use it).
def load_corpus(corpus_file):
    with open(corpus_file, "r", encoding="utf-8") as f:
        data = json.load(f)
    passages = [entry["passage"] for entry in data if "passage" in entry]
    chunk_ids = [entry["chunk_id"] for entry in data if "chunk_id" in entry]
    print(f"Loaded {len(passages)} corpus passages")
    return passages, chunk_ids

# A helper function to load subqueries from a ground truth file.
def retrieve_all_subqueries(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        qa_data = json.load(f)
    subqueries = []
    for item in qa_data:
        subqueries.extend(item["sub_questions"])
    return subqueries

# Paths for the corpus and QA sets.
CORPUS_FILE = "../data/chunked_text_all_together_cleaned.json"
QA_PATH = "../data/QA_set"
QA_EMBEDDED_PATH = "../data/QA_set_embedded"
PM_PATH = "../performance"

# CORPUS_FILE = os.path.join(project_root, "data", "chunked_text_all_together_cleaned.json")
# QA_PATH = os.path.join(project_root, "data", "QA_set")
# QA_EMBEDDED_PATH = os.path.join(project_root, "data", "QA_set_embedded")
# Load the corpus and build common indexes.

passages, chunk_ids = load_corpus(CORPUS_FILE)

# Build TF-IDF index for the corpus.
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
doc_matrix = tfidf_vectorizer.fit_transform(passages)

# Build BM25 index for the corpus.
tokenized_passages = [word_tokenize(p.lower()) for p in passages]
bm25 = BM25Okapi(tokenized_passages)

# Load global dense corpus index (FAISS).
corpus_dense_index = faiss.read_index("../hp_all_bge.index")

from sentence_transformers import CrossEncoder

# To refine the results, we use a CrossEncoder. A CrossEncoder gets both inputs (input_question, retrieved_question)
# and outputs a score 0...1 indicating the similarity.
cross_encoder_model = CrossEncoder("cross-encoder/stsb-roberta-base")

Loaded 9251 corpus passages


In [ ]:

# Define dataset configurations.
# For each dataset, specify:
#  - "name": a short name,
#  - "gt_path": path to the ground truth QA file,
#  - For dense retrieval: "subqueries" (list) and "subquery_index" (FAISS index file path).
datasets = [
    {
        "name": "easy_single",
        "gt_path": os.path.join(QA_PATH, "easy_single_labeled.json"),
        "dense": {
            "subqueries": retrieve_all_subqueries(os.path.join(QA_PATH, "easy_single_labeled.json")),
            "subquery_index": faiss.read_index(os.path.join(QA_EMBEDDED_PATH, "bge_easy_single_labeled.index"))
        }
    },
    {
        "name": "medium_single",
        "gt_path": os.path.join(QA_PATH, "medium_single_labeled.json"),
        "dense": {
            "subqueries": retrieve_all_subqueries(os.path.join(QA_PATH, "medium_single_labeled.json")),
            "subquery_index": faiss.read_index(os.path.join(QA_EMBEDDED_PATH, "bge_medium_single_labeled.index"))
        }
    },
    {
        "name": "medium_multi",
        "gt_path": os.path.join(QA_PATH, "medium_multi_labeled.json"),
        "dense": {
            "subqueries": retrieve_all_subqueries(os.path.join(QA_PATH, "medium_multi_labeled.json")),
            "subquery_index": faiss.read_index(os.path.join(QA_EMBEDDED_PATH, "bge_medium_multi_labeled.index"))
        }
    },
    {
        "name": "hard_single",
        "gt_path": os.path.join(QA_PATH, "hard_single_labeled.json"),
        "dense": {
            "subqueries": retrieve_all_subqueries(os.path.join(QA_PATH, "hard_single_labeled.json")),
            "subquery_index": faiss.read_index(os.path.join(QA_EMBEDDED_PATH, "bge_hard_single_labeled.index"))
        }
    },
    {
        "name": "hard_multi",
        "gt_path": os.path.join(QA_PATH, "hard_multi_labeled.json"),
        "dense": {
            "subqueries": retrieve_all_subqueries(os.path.join(QA_PATH, "hard_multi_labeled.json")),
            "subquery_index": faiss.read_index(os.path.join(QA_EMBEDDED_PATH, "bge_hard_multi_labeled.index"))
        }
    }
]


#########################################
# Retrieval Methods Configuration
#########################################

# Create a dictionary mapping retrieval method names to their functions.
retrieval_methods = {
    "tfidf": tfidf_retrieval,
    "bm25": bm25_retrieval,
    "dense": dense_retrieval_subqueries,
    "tfidf_dense": hybrid,
    "bm25_dense": hybrid,
    "dense_tfidf": hybrid,
    "dense_bm25": hybrid,
    "cross_encoder": hybrid
    
}

# For each dataset, define a configuration for each retrieval method.
# Here we create a dictionary keyed by retrieval method for each dataset.
for ds in datasets:
    ds.setdefault("retrieval_config", {})
    #TF-IDF configuration.
    ds["retrieval_config"]["tfidf"] = {
        "passages": passages,
        "chunk_ids": chunk_ids,
        "vectorizer": tfidf_vectorizer,
        "doc_matrix": doc_matrix,
        "top_k": 20,
        "query_type": "question"  # or "sub_questions" based on your data structure
    }
    # BM25 configuration.
    ds["retrieval_config"]["bm25"] = {
        "passages": passages,
        "chunk_ids": chunk_ids,
        "bm25": bm25,
        "top_k": 20,
        "query_type": "question"  # or "sub_questions" based on your data structure
    }
    #Dense retrieval configuration.
    ds["retrieval_config"]["dense"] = {
        "passages": passages,
        "chunk_ids": chunk_ids,
        "all_subqueries": ds["dense"]["subqueries"],
        "subquery_index": ds["dense"]["subquery_index"],
        "faiss_index": corpus_dense_index,
        "top_k": 20,
        "query_type": "sub_questions"  # or "question" based on your data structure
    }
        # --- Add Hybrid Configurations ---
    # Hybrid TF-IDF + Dense configuration.
    ds["retrieval_config"]["tfidf_dense"] = {
        "sparse_func": tfidf_retrieval,
        "sparse_config": ds["retrieval_config"]["tfidf"],
        "dense_func": dense_retrieval_subqueries,
        "dense_config": ds["retrieval_config"]["dense"],
        "intermediate_k": 1000,   # You can choose this value independently.
        "final_k": 20,           # And choose the final number of results.
        "mode": "intersection" , # or "union"
        "query_type": "combine",  # or "sub_questions" based on your data structure
        "order": "sparse_dense"  # or "dense_sparse"

    }
    # Hybrid BM25 + Dense configuration.
    ds["retrieval_config"]["bm25_dense"] = {
        "sparse_func": bm25_retrieval,
        "sparse_config": ds["retrieval_config"]["bm25"],
        "dense_func": dense_retrieval_subqueries,
        "dense_config": ds["retrieval_config"]["dense"],
        "intermediate_k": 1000,
        "final_k": 20,
        "mode": "intersection" , # or "union"
        "query_type": "combine",  # or "sub_questions" based on your data structure
        "order": "sparse_dense"  # or "dense_sparse"
    }
    ds["retrieval_config"]["dense_tfidf"] = {
        "sparse_func": tfidf_retrieval,
        "sparse_config": ds["retrieval_config"]["tfidf"],
        "dense_func": dense_retrieval_subqueries,
        "dense_config": ds["retrieval_config"]["dense"],
        "intermediate_k": 1000,   # You can choose this value independently.
        "final_k": 20,           # And choose the final number of results.
        "mode": "intersection" , # or "union"
        "query_type": "combine",  # or "sub_questions" based on your data structure
        "order": "dense_sparse"  # or "dense_sparse"

    }
    # Hybrid BM25 + Dense configuration.
    ds["retrieval_config"]["dense_bm25"] = {
        "sparse_func": bm25_retrieval,
        "sparse_config": ds["retrieval_config"]["bm25"],
        "dense_func": dense_retrieval_subqueries,
        "dense_config": ds["retrieval_config"]["dense"],
        "intermediate_k": 1000,
        "final_k": 20,
        "mode": "intersection" , # or "union"
        "query_type": "combine",  # or "sub_questions" based on your data structure
        "order": "dense_sparse"  # or "dense_sparse"
    }
    # Hybrid with Cross Encoder configuration.
    ds["retrieval_config"]["cross_encoder"] = {
        "sparse_func": tfidf_retrieval,
        "sparse_config": ds["retrieval_config"]["tfidf"],
        "dense_func": dense_retrieval_subqueries,
        "dense_config": ds["retrieval_config"]["dense"],
        "intermediate_k": 30,
        "final_k": 5,
        "mode": "intersection",
        "rerank": True,
        "cross_encoder_model": cross_encoder_model
    }


#########################################
# Test Process: Evaluate Each Retrieval Method on Each Dataset
#########################################
# load the results
save_path = os.path.join(QA_PATH, "retrieval_results.json")
with open(save_path, "r") as f:
    results = json.load(f)

# can select the datasets you want to evaluate and the retrieval methods you want to evaluate
select_datasets = ["easy_single", "medium_single"]
select_retrieval_methods = ["cross_encoder"]
# Filter datasets based on the selected names.
datasets = [ds for ds in datasets if ds["name"] in select_datasets]
# Filter retrieval methods based on the selected names.
retrieval_methods = {k: v for k, v in retrieval_methods.items() if k in select_retrieval_methods}
for ds in datasets:
    # Load ground truth data.
    with open(ds["gt_path"], "r", encoding="utf-8") as f:
        ground_truth_data = json.load(f)
    print(f"Dataset '{ds['name']}': loaded {len(ground_truth_data)} ground truth examples.")
    
    # For each retrieval method defined in this dataset's config:
    for method_name, method_func in retrieval_methods.items():
        # Check if a configuration exists for this method.
        if method_name in ds["retrieval_config"]:
            config = ds["retrieval_config"][method_name]
            predictions = []
            references = []
            for example in ground_truth_data:
                query_type = config.get("query_type")
                query = [example["question"], example["sub_questions"]]
                if query_type == "sub_questions":
                    question = query[1]
                elif query_type == "question":
                    question = query[0]
                else:
                    question = query 
                ret_results = method_func(question, config)
                # Collect predicted chunk IDs (convert to string).
                pred_ids = [str(r["chunk_id"]) for r in ret_results]
                predictions.append(pred_ids)
                # Ground truth: assume each reference in "list of reference" has a "ref_id".
                gt_ids = [str(ref["ref_id"]) for ref in example["list of reference"]]
                references.append(gt_ids)
            
            # Initialize evaluation metrics.
            eval_collection = MetricCollection({
                "recall@5": RecallAtK(k=5),
                "recall@10": RecallAtK(k=10),
                "precision@5": PrecisionAtK(k=5),
                "precision@10": PrecisionAtK(k=10),
                "mrr": MRR(),
                "ndcg@5": nDCG(k=5),
                "ndcg@10": nDCG(k=10)
            })
            eval_collection.update(predictions, references, metric_type="retrieval")
            overall_metrics = eval_collection.compute(metric_type="retrieval")
            
            results.append({
                "data_set": ds["name"],
                "retrieval": method_name,
                "performance": overall_metrics
            })
            print(f"Evaluated {method_name} on {ds['name']}: {overall_metrics}")

#########################################
# Now 'results' contains evaluation metrics for each dataset and each retrieval method.
#########################################

# Optionally, you can now plot the results. For example, a grouped bar chart comparing methods on each dataset.
# (You can adapt your previous plotting code using the results list.)
print('get the results, if you want to save the results, please go to next cell')


Dataset 'easy_single': loaded 253 ground truth examples.
Error during cross encoder prediction: CUDA error: device-side assert triggered
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.

Error during cross encoder prediction: CUDA error: device-side assert triggered
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.

Error during cross encoder prediction: CUDA error: device-side assert triggered
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.

Error during cross encoder prediction: CUDA error: device-side assert triggered
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.

Error during cross encoder prediction: CUDA error: device-side assert triggered
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.

Error during cross encoder prediction: CUDA error: device-side assert triggered
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.

Error during cross encoder prediction: CUDA error

ValueError: Query 'What is the name of the spell Harry uses to disarm Draco Malfoy? How does Harry's spell work in the context of the Room of Requirement? Is this spell a defensive or offensive incantation?' not found in the list of all queries.

env: CUDA_LAUNCH_BLOCKING=1


In [8]:

PM_PATH = "../performance"
# save the results to a JSON file.
save_path = os.path.join(PM_PATH, "retrieval_performance_results.json")
# Ensure the directory exists.
os.makedirs(os.path.dirname(save_path), exist_ok=True)
# Save the results to the specified JSON file.
with open(save_path, "w") as f:
    json.dump(results, f, indent=2)
# Print a message indicating where the results were saved.
print(f"Results saved to {save_path}")

Results saved to ../performance\retrieval_performance_results.json
